In [2]:
import maup # mggg's library for proration, see documentation here: https://github.com/mggg/maup
import pandas as pd # standard python data library
import geopandas as gp # the geo-version of pandas
import numpy as np 
import os
import fiona
from statistics import mean, median
from pandas import read_csv
gp.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw' #To load KML files
import string

Georgia
-------
Election results from the Georgia Secretary of State Elections Division (https://sos.ga.gov/index.php/Elections/current_and_past_elections_results)
Precinct shapefile primarily from the Georgia General Assembly Reapportionment Office (http://www.legis.ga.gov/Joint/reapportionment/en-US/default.aspx)

2018 Election Results by Precinct

Vote Column Label Format
------------------------
Columns reporting votes follow a standard label pattern. One example is:
G16PREDCli
The first character is G for a general election, P for a primary, S for a special, and R for a runoff.
Characters 2 and 3 are the year of the election.
Characters 4-6 represent the office type (see list below).
Character 7 represents the party of the candidate.
Characters 8-10 are the first three letters of the candidate's last name.

Office Codes
A## - Ballot amendment, where ## is an identifier
AGR - Commissioner of Agriculture
ATG - Attorney General
AUD - Auditor
CFO - Chief Financial Officer
CHA - Council Chairman
COC - Corporation Commissioner
COM - Comptroller
CON - State Controller
COU - City Council Member
CSC - Clerk of the Supreme Court
DEL - Delegate to the U.S. House
GOV - Governor
H## - U.S. House, where ## is the district number. AL: at large.
HOD - House of Delegates, accompanied by a HOD_DIST column indicating district number
HOR - U.S. House, accompanied by a HOR_DIST column indicating district number
INS - Insurance Commissioner
LAB - Labor Commissioner
LND - Commissioner of Public/State Lands
LTG - Lieutenant Governor
MAY - Mayor
MNI - State Mine Inspector
PSC - Public Service Commissioner
PUC - Public Utilities Commissioner
RGT - State University Regent
SAC - State Appeals Court
SBE - State Board of Education
SOC - Secretary of Commonwealth
SOS - Secretary of State
SPI - Superintendent of Public Instruction
SPL - Commissioner of School and Public Lands
SSC - State Supreme Court
TAX - Tax Commissioner
TRE - Treasurer
UBR - University Board of Regents/Trustees/Governors
USS - U.S. Senate

Party Codes
D and R will always represent Democrat and Republican, respectively.
See the state-specific notes for the remaining codes used in a particular file; note that third-party candidates may appear on the ballot under different party labels in different states.

Fulton County uses shapefiles and maps sourced from the county GIS program. Precincts CH01/CH04B, CP07A/CP07D, SS29A/SS29B, UC031/UC035 were merged to match how voters were registered in the 2018 voter file.  

The following precincts were split by congressional district to match the 2018 election results: Avondale High, Glennwood, Wadsworth in Dekalb County; Cates D in Gwinnett County.  

Cloudland and Teloga precincts in Chattooga County were split along the ridgeline that marks the boundary between them with the USGS Topographic Contour shapefile.  

Three of the four VTDs in Chattahoochee County are comprised of Fort Benning. However, the county only has one polling location for all voters, including residents of Fort Benning that vote within the county. The four Chattahoochee County VTDs have therefore been merged in the shapefile.  

G18GOVRKEM - Brian Kemp (Republican Party)  
G18GOVDABR - Stacey Abrams (Democratic Party)  
G18GOVLMET - Ted Metz (Libertarian Party)  

G18LTGRDUN - Geoff Duncan (Republican Party)  
G18LTGDAMI - Sarah Riggs Amico (Democratic Party)  

G18SOSRRAF - Brad Raffensperger (Republican Party)  
G18SOSDBAR - John Barrow (Democratic Party)  
G18SOSLDUV - Smythe Duval (Libertarian Party)  

G18ATGRCAR - Chris Carr (Republican Party)  
G18ATGDBAI - Charlie Bailey (Democratic Party)  

G18AGRRBLA - Gary Black (Republican Party)  
G18AGRDSWA - Fred Swann (Democratic Party)  

G18INSRBEC - Jim Beck (Republican Party)  
G18INSDLAW - Janice Laws (Democratic Party)  
G18INSLFOS - Donnie Foster (Libertarian Party)  

G18SPIRWOO - Richard Woods (Republican Party)  
G18SPIDTHO - Otha E. Thornton, Jr. (Democratic Party)  

G18LABRBUT - Mark Butler (Republican Party)  
G18LABDKEA - Richard Keatley (Democratic Party)  

G18PSCREAT - Chuck Eaton (Republican Party)  
G18PSCDMIL - Lindy Miller (Democratic Party)  
G18PSCLGRA - Ryan Graham (Libertarian Party)  

G18PSCRPRI - Tricia Pridemore (Republican Party)  
G18PSCDRAN - Dawn A. Randolph (Democratic Party)  
G18PSCLTUR - John Turpish (Libertarian Party)  

R18SOSRRAF - Brad Raffensperger (Republican Party)  
R18SOSDBAR - John Barrow (Democratic Party)  

R18PSCREAT - Chuck Eaton (Republican Party)  
R18PSCDMIL - Lindy Miller (Democratic Party)  

In [11]:
vest_ga_18 = gp.read_file("./raw-from-source/VEST/ga_2018/ga_2018.shp")

In [28]:
print(vest_ga_18.shape)
print(vest_ga_18.crs)
print(vest_ga_18["CTYSOSID"].value_counts())
print(vest_ga_18[vest_ga_18["CTYSOSID"]==""])

(2659, 38)
epsg:4019
135035    2
089AI     2
089WA     2
089GA     2
245802    1
         ..
057041    1
089MS     1
135100    1
161502    1
13704     1
Name: CTYSOSID, Length: 2647, dtype: int64
Empty GeoDataFrame
Columns: [DISTRICT, CTYSOSID, PRECINCT_I, PRECINCT_N, CTYNAME, CTYNUMBER, CTYNUMBER2, FIPS2, G18GOVRKEM, G18GOVDABR, G18GOVLMET, G18LTGRDUN, G18LTGDAMI, G18SOSRRAF, G18SOSDBAR, G18SOSLDUV, G18ATGRCAR, G18ATGDBAI, G18AGRRBLA, G18AGRDSWA, G18INSRBEC, G18INSDLAW, G18INSLFOS, G18SPIRWOO, G18SPIDTHO, G18LABRBUT, G18LABDKEA, G18PSCREAT, G18PSCDMIL, G18PSCLGRA, G18PSCRPRI, G18PSCDRAN, G18PSCLTUR, R18SOSRRAF, R18SOSDBAR, R18PSCREAT, R18PSCDMIL, geometry]
Index: []

[0 rows x 38 columns]


In [13]:
print(vest_ga_18.head())

  DISTRICT CTYSOSID PRECINCT_I PRECINCT_N CTYNAME CTYNUMBER CTYNUMBER2 FIPS2  \
0  121JC15  121JC15       JC15       JC15  FULTON        60        060   121   
1  12108N1  12108N1       08N1       08N1  FULTON        60        060   121   
2  12108N2  12108N2       08N2       08N2  FULTON        60        060   121   
3  121JC14  121JC14       JC14       JC14  FULTON        60        060   121   
4  121JC05  121JC05       JC05       JC05  FULTON        60        060   121   

   G18GOVRKEM  G18GOVDABR  ...  G18PSCDMIL  G18PSCLGRA  G18PSCRPRI  \
0         193         309  ...         295          30         198   
1         349         224  ...         198          14         368   
2         399         510  ...         468          23         420   
3         910         704  ...         643          36         924   
4         592         763  ...         723          48         593   

   G18PSCDRAN  G18PSCLTUR  R18SOSRRAF  R18SOSDBAR  R18PSCREAT  R18PSCDMIL  \
0         289        

In [14]:
precincts_ga = gp.read_file("./raw-from-source/GA_shapefile/vtd2018-shapefile/VTD2018-Shapefile.shp")

In [22]:
print(precincts_ga.shape)
print(precincts_ga.crs)
print(precincts_ga[precincts_ga["CTYSOSID"]==None])

(2660, 12)
epsg:4019
Empty GeoDataFrame
Columns: [ID, AREA, DATA, DISTRICT, CTYSOSID, PRECINCT_I, PRECINCT_N, CTYNAME, CTYNUMBER, CTYNUMBER2, FIPS2, geometry]
Index: []


In [24]:
test = pd.merge(vest_ga_18,precincts_ga,on="CTYSOSID",how="inner",indicator=True)

In [25]:
print(test["_merge"].value_counts())

both          2873
right_only       0
left_only        0
Name: _merge, dtype: int64


In [23]:
print(test.shape)

(2893, 50)
